In [5]:
from analysis_tools.clustering import compute_scores_sym_matrix_fast, compute_volumes_pockets,compute_partial_scores_matrix_fast
from analysis_tools.clustering import compute_clusters
from sklearn import cluster
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, fcluster,linkage
from scipy.cluster.hierarchy import set_link_color_palette

import numpy as np
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix

from sklearn.cluster import OPTICS
import matplotlib.pyplot as plt
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.manifold import MDS
import math

from pyminimax import minimax

Compute scores matrix (Spectral descriptors and Zernike descriptors)

In [ ]:
#Find example data of scores and entries in /pdb_shape_retrieval/example_data
#monomers

scores_file = '/Users/gdiazleines/results/shape-retrieval/benchmarking/Spike_protein/analysis/molstar_medium-resolution/pairs_scores.txt'
entries_file = '/Users/gdiazleines/results/shape-retrieval/benchmarking/Spike_protein/analysis/molstar_medium-resolution/list_entries.txt'

#compute scores matrix
scores_matrix, labels = compute_partial_scores_matrix_fast(scores_file, entries_file)
scores_matrix=scores_matrix/scores_matrix.max()
#scores_matrix=scores_matrix/55.0

#plot scores matrix
fig, (ax1) = plt.subplots(1, figsize=(4, 4))
sns.heatmap(scores_matrix,ax=ax1,vmin=0.0,cmap ='seismic')


Combine Zernike and Spectrals similarity scores and Compute scores matrix 

In [ ]:
#Find example data of scores and entries in /pdb_shape_retrieval/example_data

scores_spectral_file = '/Users/gdiazleines/results/shape-retrieval/benchmarking/bacterial_ribosome/shape_analysis/pairs_scores.txt'
entries_file = '/Users/gdiazleines/results/shape-retrieval/benchmarking/bacterial_ribosome/shape_analysis/list_entries.txt'
scores_zernike_file ='/Users/gdiazleines/results/shape-retrieval/benchmarking/bacterial_ribosome/shape_analysis/zernike_data/fullatom_prediction.txt'


#compute scores matrix
scores_spectral_matrix, labels = compute_partial_scores_matrix_fast(scores_spectral_file, entries_file)
scores_zernike_matrix, labels = compute_partial_scores_matrix_fast(scores_zernike_file, entries_file)

scores_zernike_matrix=scores_zernike_matrix/scores_zernike_matrix.max()
scores_spectral_matrix=scores_spectral_matrix/scores_spectral_matrix.max()

#combine scores
scores_matrix = (scores_spectral_matrix*scores_zernike_matrix)**0.5

#plot scores matrix
fig, (ax1) = plt.subplots(1, figsize=(4, 4))
#sns.heatmap(sym_matrix,ax=ax1,vmin=0.0,cmap ='seismic',xticklabels=labels,yticklabels=labels)
sns.heatmap(scores_matrix,ax=ax1,vmin=0.0,cmap ='seismic')


Hierarchical clustering

In [ ]:
#Define a threshold or number of clusters
threshold = None
n_clusters=3

#define clustering method ('ward' or 'average') and compute clusters
linkage_method="ward"

#Compute clusters
clusters,n_clusters,link_matrix,threshold_dist = compute_clusters(scores_matrix,labels,cluster,linkage_method, threshold, n_clusters)

print('optimal number of clusters',n_clusters)
print('threshold distance',threshold_dist)

#set clustering colour palette
set_link_color_palette(['red', 'blue', 'green', 'orange', 'purple','black'])

#plot dendogram
dendrogram(link_matrix,truncate_mode = "level", p=50,color_threshold=threshold_dist, labels=labels, above_threshold_color='lightgrey')
plt.axhline(threshold_dist, color='k', linestyle='--')
plt.xticks(rotation=90)  # Rotate labels vertically
plt.tight_layout() 
plt.show()

Minimax clustering

In [ ]:
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist

Z = minimax(pdist(scores_matrix))

max_clusters = 4
clusters = fcluster(Z, t=max_clusters, criterion='maxclust')

# Step 4: Print clusters with labels
for cluster_id in sorted(set(clusters)):
    members = [label for label, c in zip(labels, clusters) if c == cluster_id]
    print(f"Cluster {cluster_id}: {members}")

fig = plt.figure(figsize=(10, 4))
dendrogram(Z, labels=labels)
plt.xticks(rotation=90)  # Rotate labels vertically
plt.tight_layout() 
plt.show()

OPTICS clustering

In [ ]:

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
D_scaled = scores_matrix

optics = OPTICS(min_samples=2, xi=0.05)
optics.fit(D_scaled)

# 4️⃣ Get cluster labels (automatic, no eps)
cluster_labels = optics.labels_
print("Cluster labels length:", len(cluster_labels))  # should be 29
print("Cluster labels:", cluster_labels)

# 5️⃣ Embed distances in 2D for plotting
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
coords = mds.fit_transform(scores_matrix)

# 6️⃣ Plot clusters
plt.figure(figsize=(5, 5))
unique_clusters = np.unique(cluster_labels)
colors = plt.cm.tab10(np.linspace(0, 1, len(unique_clusters)))

for cluster_id, color in zip(unique_clusters, colors):
    mask = cluster_labels == cluster_id
    plt.scatter(coords[mask, 0], coords[mask, 1],
                c='k' if cluster_id == -1 else color,
                s=120,
                marker='x' if cluster_id == -1 else 'o',
                label='Noise' if cluster_id == -1 else f'Cluster {cluster_id}')

# Add point labels
for i, (x, y) in enumerate(coords):
    plt.text(x + 0.3, y + 0.3, labels[i], fontsize=4)

# Auto-pad axes
plt.xlim(coords[:,0].min() - 1, coords[:,0].max() + 1)
plt.ylim(coords[:,1].min() - 1, coords[:,1].max() + 1)

plt.title('OPTICS Clustering (automatic, xi=0.1)')
plt.xlabel('MDS Dimension 1')
plt.ylabel('MDS Dimension 2')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()




In [ ]:
volumes_file = '/Users/gdiazleines/results/shape-retrieval/benchmarking/JandJ/targets_results/lipase/rep_2/pockets_r_volume.txt'

pockets_volumes = []
#pocket_volumes= compute_volumes_pockets(volumes_file,clusters)
#print(clusters)
for i in range(len(clusters)):
    pocket_volumes= compute_volumes_pockets(volumes_file,clusters[i])
    for pocket in pocket_volumes:
        print(i,pocket[0],pocket[1])
